In [19]:
from extact_model import *
import numpy as np
import json
from tqdm import tqdm

In [21]:
data = load_data('sfzy_small_extract.json')
data_x = np.load('vector_exteact.npy')
input_size = 768
hidden_size = 384
epochs = 20
batch_size = 64
threshold = 0.2
num_folds = 15
max_len = 256
model = bulid_extract_model(max_len,input_size,hidden_size)

def pred_one_fold(data,data_x,model,fold,num_folds):
    valid_data = data_split(data, fold, num_folds, 'valid')
    valid_x = data_split(data_x, fold, num_folds, 'valid')
    model.load_weights('./weights/extract_model_%s.hdf5' % fold)
    y_pred = model.predict(valid_x)[:,:,0]
    results = []
    for d, yp in tqdm(zip(valid_data, y_pred), desc=u'转换中'):
        yp = yp[:len(d[0])]
        yp = np.where(yp > 0.2)[0]
        source_1 = ''.join([d[0][i] for i in yp])
        source_2 = ''.join([d[0][i] for i in d[1]])
        result = {
            'source_1' : source_1,
            'source_2' : source_2,
            'target' : d[2]
        }
        results.append(result)
    return results

In [22]:
def convert(filename, data,data_x,model,num_folds):
    """转换为生成式数据
    """
    F = open(filename, 'w', encoding='utf-8')
    total_results = []
    for fold in tqdm(range(num_folds),desc='folding'):
        total_results.append(pred_one_fold(data,data_x,model,fold,num_folds))
    n = 0
    while True:
        i, j = n % num_folds, n // num_folds
        try:
            d = total_results[i][j]
        except:
            break
        F.write(json.dumps(d, ensure_ascii=False) + '\n')
        n += 1
    F.close()

In [23]:
convert('sfzy_seq2seq.json',data,data_x,model,num_folds)

folding:   0%|          | 0/15 [00:00<?, ?it/s]
转换中: 270it [00:00, 39802.55it/s]
folding:   7%|▋         | 1/15 [00:04<00:57,  4.11s/it]
转换中: 270it [00:00, 43770.03it/s]
folding:  13%|█▎        | 2/15 [00:04<00:40,  3.12s/it]
转换中: 270it [00:00, 43858.18it/s]
folding:  20%|██        | 3/15 [00:05<00:29,  2.46s/it]
转换中: 270it [00:00, 44873.09it/s]
folding:  27%|██▋       | 4/15 [00:06<00:21,  1.95s/it]
转换中: 270it [00:00, 44823.36it/s]
folding:  33%|███▎      | 5/15 [00:07<00:16,  1.61s/it]
转换中: 270it [00:00, 43618.31it/s]
folding:  40%|████      | 6/15 [00:08<00:12,  1.36s/it]
转换中: 270it [00:00, 46824.98it/s]
folding:  47%|████▋     | 7/15 [00:08<00:09,  1.17s/it]
转换中: 270it [00:00, 48340.04it/s]
folding:  53%|█████▎    | 8/15 [00:09<00:07,  1.05s/it]
转换中: 270it [00:00, 43240.25it/s]
folding:  60%|██████    | 9/15 [00:10<00:05,  1.04it/s]
转换中: 270it [00:00, 48996.76it/s]
folding:  67%|██████▋   | 10/15 [00:11<00:04,  1.08it/s]
转换中: 270it [00:00, 45396.54it/s]
folding:  73%|███████▎  | 11

In [24]:
with open('sfzy_seq2seq.json', encoding='utf-8') as f:
    for l in f:
        l = json.loads(l)
        print(l)
        break

{'source_1': '原告玄兆甲与被告黑龙江龙贸房地产综合开发有限责任公司（以下简称龙贸房地产公司）、南星、金南借款合同纠纷一案玄兆甲向本院提出诉讼请求1、被告给付原告借款本金1,600,000元及利息（自2013年8月15日起至实际给付之日止被告龙贸房地产公司辩称本案已经超过诉讼时效玄兆甲与龙贸房地产公司签订《借款合同》合法有效双方债权、债务关系明确龙贸房地产公司作为债务人应当履行偿还全部借款及利息的义务而其未于玄兆甲催告还款后偿还借款应当依法支付逾期利息因双方未约定还款期限玄兆甲在知道权利已被侵害时诉至法院主张债权依照《中华人民共和国民法通则》第一百三十五条、第一百三十七条、第一百四十条《中华人民共和国合同法》第六十条第一款、第二百零五条、第二百零六条《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》第二十六条、第二十九条之规定判决如下一、被告黑龙江龙贸房地产综合开发有限责任公司于本判决生效之日立即偿还原告玄兆甲借款本金1,600,000元二、被告黑龙江龙贸房地产综合开发有限责任公司于本判决生效之日立即给付原告玄兆甲自2013年11月6日起至实际给付之日止如果未按本判决指定的期间履行给付金钱义务加倍支付迟延履行期间的债务利息', 'source_2': '被告并未还款故原告诉至法院故对玄兆甲要求龙贸房地产公司偿还借款本金1,600,000元的诉讼请求关于龙贸房地产公司辩称已经偿还玄兆甲借款利息288,000元而玄兆甲认可其偿还的借款利息为264,000元因双方约定的利率超过国家法律规定的上限依照《中华人民共和国民法通则》第一百三十五条、第一百三十七条、第一百四十条《中华人民共和国合同法》第六十条第一款、第二百零五条、第二百零六条《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》第二十六条、第二十九条之规定判决如下一、被告黑龙江龙贸房地产综合开发有限责任公司于本判决生效之日立即偿还原告玄兆甲借款本金1,600,000元按照年利率24%计算的上述借款利息', 'target': '原被告系借款合同纠纷。原告提出诉请，要求被告偿还借款160万元借款本息。被告辩称：1、原告预先扣除利息，实际借款金额应为1504000元；2、本案已超过诉讼请求；3、双方约定利率过高，原告应返还相应利息。经查明：1、原被告借款合同合法有效；2、借款金额为160万元；3